Возьмите датасет EMNIST из torchvision.
Обучите на нём модели: ResNet 18, VGG 16, Inception v3, DenseNet 161:
желательно обучить каждую модель с нуля по 10 эпох
если ресурсов компьютера / Colab не хватает, достаточно обучить каждую модель по 1-2 эпохи
Сведите результаты обучения моделей (графики лосса) в таблицу и сравните их.
Дополнительное задание (выполняется по желанию)
Выполните то же задание, используя датасет hymenoptera_data

ResNet 18

In [ ]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()# перевод модели в режим теста а не обучение
    for X, y in data_iter:
        X, y = X.to(device), y.to(device) # передаем на устройство
        acc_sum += (net(X).argmax(axis=1) == y).sum() # считаем сумму
        n += y.shape[0]
    return acc_sum.item()/n



In [ ]:
acc =[]

def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum') # ищем суммарно накопленную ошибку
    net.train() # переводим сеть в режим обучения
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        curr_acc = 0
        for i, (X, y) in enumerate(train_iter):

            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X) # пердсказание
            l = loss(y_hat, y) # считаем ошибку на предсказаниях
            l.backward() # делаем подсчет градиентов
            trainer.step() # делаем шаг градиентного спуска
            train_l_sum += l.item() # накаплиаем ошибку
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()# смотрить сколько раз отвечали правильно
            n += y.shape[0]# смотрим кол-воб объектов которые нейросеть увидела.

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        curr_acc = test_acc
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')
    acc.append(curr_acc)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
BATCH_SIZE = 256
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),# у нас изображение чб,а сеть принимает цветное. Для этого надо увеличить канальность серого
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
summary(model.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
# Убираем требование градиента:
# т.е. делаем веса не обучаемыми
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc# здесь у нас на выходе 1000 сигнаов. Надо 10

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=10) # меняем на 10

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True: # в новом слое величина равна True. его параметры и будем длбавлять в список.
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train(model, train_iter, test_iter, trainer, 1)

Step 0. time since epoch: 0.889. Train acc: 0.934. Train Loss: 0.260
Step 10. time since epoch: 8.713. Train acc: 0.941. Train Loss: 0.261
Step 20. time since epoch: 14.970. Train acc: 0.944. Train Loss: 0.252
Step 30. time since epoch: 21.856. Train acc: 0.941. Train Loss: 0.257
Step 40. time since epoch: 28.092. Train acc: 0.940. Train Loss: 0.260
Step 50. time since epoch: 34.957. Train acc: 0.940. Train Loss: 0.258
Step 60. time since epoch: 41.137. Train acc: 0.940. Train Loss: 0.257
Step 70. time since epoch: 47.965. Train acc: 0.940. Train Loss: 0.255
Step 80. time since epoch: 54.155. Train acc: 0.940. Train Loss: 0.250
Step 90. time since epoch: 60.965. Train acc: 0.941. Train Loss: 0.248
Step 100. time since epoch: 67.167. Train acc: 0.941. Train Loss: 0.246
Step 110. time since epoch: 74.142. Train acc: 0.941. Train Loss: 0.245
Step 120. time since epoch: 80.332. Train acc: 0.940. Train Loss: 0.245
Step 130. time since epoch: 87.129. Train acc: 0.940. Train Loss: 0.245
Step 

In [ ]:
acc

[0.9511]

VGG 16

In [ ]:
BATCH_SIZE = 256
transoforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)), # переделываем картинки  размер 224 на 224
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE) # за 1 итерацию выдает батч размером BATCH_SIZE
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
def vgg_block(num_convs, input_channels, num_channels):
    blk = nn.Sequential(nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1), nn.ReLU())
    for i in range(num_convs - 1):
        blk.add_module("conv{}".format(i), nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1))
        blk.add_module("relu{}".format(i), nn.ReLU())
    blk.add_module("pool", nn.MaxPool2d(2, stride=2))
    return blk

In [ ]:
def vgg(conv_arch):
    net = nn.Sequential()

    for i, (num_convs, input_ch, num_channels) in enumerate(conv_arch):
        net.add_module("block{}".format(i), vgg_block(num_convs, input_ch, num_channels))


    classifier = nn.Sequential(
        nn.Flatten(),

        # nn.Linear(25088, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(6272, 4096), nn.ReLU(), nn.Dropout(0.5),

        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

    net.add_module('classifier', classifier)
    return net

In [ ]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))

In [ ]:
ratio = 4 # уменьшим канальности в 4 раза чтобы сеть быстрее обучалась и вмещалась в видеопамять.
small_conv_arch = [(v[0], max(v[1] // ratio, 1), v[2] // ratio) for v in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             160
              ReLU-2         [-1, 16, 224, 224]               0
         MaxPool2d-3         [-1, 16, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]           4,640
              ReLU-5         [-1, 32, 112, 112]               0
         MaxPool2d-6           [-1, 32, 56, 56]               0
            Conv2d-7           [-1, 64, 56, 56]          18,496
              ReLU-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          36,928
             ReLU-10           [-1, 64, 56, 56]               0
        MaxPool2d-11           [-1, 64, 28, 28]               0
           Conv2d-12          [-1, 128, 28, 28]          73,856
             ReLU-13          [-1, 128, 28, 28]               0
           Conv2d-14          [-1, 128,

In [ ]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 0.514. Train acc: 0.094. Train Loss: 2.303
Step 10. time since epoch: 5.037. Train acc: 0.110. Train Loss: 2.304
Step 20. time since epoch: 9.559. Train acc: 0.140. Train Loss: 2.272
Step 30. time since epoch: 14.586. Train acc: 0.199. Train Loss: 2.200
Step 40. time since epoch: 19.115. Train acc: 0.302. Train Loss: 1.946
Step 50. time since epoch: 23.886. Train acc: 0.390. Train Loss: 1.707
Step 60. time since epoch: 28.621. Train acc: 0.462. Train Loss: 1.512
Step 70. time since epoch: 33.193. Train acc: 0.523. Train Loss: 1.347
Step 80. time since epoch: 38.145. Train acc: 0.573. Train Loss: 1.208
Step 90. time since epoch: 42.723. Train acc: 0.613. Train Loss: 1.098
Step 100. time since epoch: 47.290. Train acc: 0.645. Train Loss: 1.009
Step 110. time since epoch: 52.368. Train acc: 0.672. Train Loss: 0.935
Step 120. time since epoch: 56.946. Train acc: 0.694. Train Loss: 0.872
Step 130. time since epoch: 61.645. Train acc: 0.714. Train Loss: 0.818
Step 1

In [ ]:
acc

[0.9511, 0.9812]

Inception v3

In [ ]:
class Inception(nn.Module):
    def __init__(self, ic, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        self.p1_1 = nn.Sequential(nn.Conv2d(ic, c1, kernel_size=1), nn.ReLU())
        self.p2_1 = nn.Sequential(nn.Conv2d(ic, c2[0], kernel_size=1), nn.ReLU())
        self.p2_2 = nn.Sequential(nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1), nn.ReLU())
        self.p3_1 = nn.Sequential(nn.Conv2d(ic, c3[0], kernel_size=1), nn.ReLU())
        self.p3_2 = nn.Sequential(nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2), nn.ReLU())
        self.p4_1 = nn.Sequential(nn.MaxPool2d(3, stride=1, padding=1))
        self.p4_2 = nn.Sequential(nn.Conv2d(ic, c4, kernel_size=1), nn.ReLU())

    def forward(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        # Concatenate the outputs on the channel dimension.
        return torch.cat((p1, p2, p3, p4), dim=1)

In [ ]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.ReLU(),
       nn.MaxPool2d(3, stride=2, padding=1))

In [ ]:
b2 = nn.Sequential(
       nn.Conv2d(64, 64, kernel_size=1),
       nn.Conv2d(64, 192, kernel_size=3, padding=1),
       nn.MaxPool2d(3, stride=2, padding=1))

In [ ]:
b3 = nn.Sequential(
       Inception(192, 64, (96, 128), (16, 32), 32),
       Inception(256, 128, (128, 192), (32, 96), 64),
       nn.MaxPool2d(3, stride=2, padding=1))

In [ ]:
b4 = nn.Sequential(
       Inception(480, 192, (96, 208), (16, 48), 64),
       Inception(512, 160, (112, 224), (24, 64), 64),
       Inception(512, 128, (128, 256), (24, 64), 64),
       Inception(512, 112, (144, 288), (32, 64), 64),
       Inception(528, 256, (160, 320), (32, 128), 128),
       nn.MaxPool2d(3, stride=2, padding=1))

In [ ]:
b5 = nn.Sequential(
       Inception(832, 256, (160, 320), (32, 128), 128),
       Inception(832, 384, (192, 384), (48, 128), 128),
       nn.AvgPool2d(7))

In [ ]:
net = nn.Sequential(b1, b2, b3, b4, b5, nn.Flatten(), nn.Linear(1024, 10))

In [ ]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           3,200
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]           4,160
            Conv2d-5          [-1, 192, 56, 56]         110,784
         MaxPool2d-6          [-1, 192, 28, 28]               0
            Conv2d-7           [-1, 64, 28, 28]          12,352
              ReLU-8           [-1, 64, 28, 28]               0
            Conv2d-9           [-1, 96, 28, 28]          18,528
             ReLU-10           [-1, 96, 28, 28]               0
           Conv2d-11          [-1, 128, 28, 28]         110,720
             ReLU-12          [-1, 128, 28, 28]               0
           Conv2d-13           [-1, 16, 28, 28]           3,088
             ReLU-14           [-1, 16,

In [ ]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 0.943. Train acc: 0.102. Train Loss: 2.303
Step 10. time since epoch: 9.908. Train acc: 0.105. Train Loss: 2.302
Step 20. time since epoch: 18.802. Train acc: 0.105. Train Loss: 2.302
Step 30. time since epoch: 27.633. Train acc: 0.109. Train Loss: 2.302
Step 40. time since epoch: 36.689. Train acc: 0.110. Train Loss: 2.302
Step 50. time since epoch: 45.818. Train acc: 0.110. Train Loss: 2.300
Step 60. time since epoch: 54.817. Train acc: 0.120. Train Loss: 2.288
Step 70. time since epoch: 64.499. Train acc: 0.125. Train Loss: 2.282
Step 80. time since epoch: 73.765. Train acc: 0.131. Train Loss: 2.263
Step 90. time since epoch: 83.704. Train acc: 0.145. Train Loss: 2.231
Step 100. time since epoch: 92.883. Train acc: 0.159. Train Loss: 2.195
Step 110. time since epoch: 102.278. Train acc: 0.173. Train Loss: 2.156
Step 120. time since epoch: 111.615. Train acc: 0.186. Train Loss: 2.121
Step 130. time since epoch: 120.877. Train acc: 0.199. Train Loss: 2.084
St

In [ ]:
acc

[0.9511, 0.9812, 0.5892]

DenseNet¶

In [ ]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),# у нас изображение чб,а сеть принимает цветное. Для этого надо увеличить канальность серого
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.densenet161(pretrained=True) # предобученная модель.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:00<00:00, 177MB/s]


In [ ]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier

Linear(in_features=2208, out_features=1000, bias=True)

In [ ]:
model.classifier = nn.Linear(in_features=2208, out_features=10) # заменяем выходной слой.

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.weight
	 classifier.bias


In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train(model, train_iter, test_iter, trainer, 1)

Step 0. time since epoch: 2.248. Train acc: 0.062. Train Loss: 2.371
Step 10. time since epoch: 24.885. Train acc: 0.435. Train Loss: 1.965
Step 20. time since epoch: 49.527. Train acc: 0.597. Train Loss: 1.674
Step 30. time since epoch: 71.907. Train acc: 0.660. Train Loss: 1.475
Step 40. time since epoch: 94.583. Train acc: 0.699. Train Loss: 1.330
Step 50. time since epoch: 117.055. Train acc: 0.730. Train Loss: 1.212
Step 60. time since epoch: 141.445. Train acc: 0.751. Train Loss: 1.122
Step 70. time since epoch: 164.524. Train acc: 0.767. Train Loss: 1.049
Step 80. time since epoch: 187.135. Train acc: 0.783. Train Loss: 0.983
Step 90. time since epoch: 209.813. Train acc: 0.795. Train Loss: 0.929
Step 100. time since epoch: 232.489. Train acc: 0.805. Train Loss: 0.881
Step 110. time since epoch: 255.205. Train acc: 0.813. Train Loss: 0.843
Step 120. time since epoch: 277.886. Train acc: 0.821. Train Loss: 0.809
Step 130. time since epoch: 300.637. Train acc: 0.827. Train Loss: 0

In [ ]:
names = ['ResNet 18', 'VGG 16', 'Inception v3', 'DenseNet 161']

In [ ]:
model_accur = zip(names, acc)
print(list(model_accur))

[('ResNet 18', 0.9511), ('VGG 16', 0.9812), ('Inception v3', 0.5892), ('DenseNet 161', 0.9304)]
